In [3]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

In [4]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import numpy as np
import calendar
import time
from matplotlib import colors
from sklearn import preprocessing
import cartopy.crs as crs
import cartopy.feature as cfeature
import scipy

from src import get_corrs
from src import format_error_df
from src import plot_heatmaps
from src import read_nwp_data
from src import forecast_error

In [10]:
df = pd.read_parquet(
    "/home/aevans/nwp_bias/src/machine_learning/data/hrrr_data/fh03/HRRR_2018_03_direct_compare_to_nysm_sites_mask_water.parquet"
)
df

index                time   latitude  \
station valid_time                                                      
ADDI    2018-03-01 03:00:00     121579 2018-03-01 00:00:00  42.019679   
        2018-03-01 04:00:00     327235 2018-03-01 01:00:00  42.019679   
        2018-03-01 05:00:00     532891 2018-03-01 02:00:00  42.019679   
        2018-03-01 06:00:00     738547 2018-03-01 03:00:00  42.019679   
        2018-03-01 07:00:00     944203 2018-03-01 04:00:00  42.019679   
...                                ...                 ...        ...   
YORK    2018-03-31 22:00:00  129672705 2018-03-31 19:00:00  42.862563   
        2018-03-31 23:00:00  129878361 2018-03-31 20:00:00  42.862563   
        2018-04-01 00:00:00  130084017 2018-03-31 21:00:00  42.862563   
        2018-04-01 01:00:00  130289673 2018-03-31 22:00:00  42.862563   
        2018-04-01 02:00:00  130495329 2018-03-31 23:00:00  42.862563   

                             longitude       t2m      sh2       d2m  \
station valid_time                                                    
ADDI    2018-03-01 03:00:00 -77.226789  7.294519  0.00613  6.003229   
        2018-03-01 04:00:00 -77.226789  7.153833  0.00618  6.086938   
        2018-03-01 05:00:00 -77.226789  7.121790  0.00620  6.110162   
        2018-03-01 06:00:00 -77.226789  6.938867  0.00615  6.033960   
        2018-03-01 07:00:00 -77.226789  6.676050  0.00605  5.809015   
...                                ...       ...      ...       ...   
YORK    2018-03-31 22:00:00 -77.833957  9.753961  0.00481  3.125024   
        2018-03-31 23:00:00 -77.833957  9.371332  0.00498  3.612512   
        2018-04-01 00:00:00 -77.833957  8.743677  0.00384 -0.051398   
        2018-04-01 01:00:00 -77.833957  7.702997  0.00433  1.622644   
        2018-04-01 02:00:00 -77.833957  7.061395  0.00458  2.411707   

                                    r2       u10       v10  ...      tcc  \
station valid_time                                          ...            
ADDI    2018-03-01 03:00:00  88.099998  3.887378  0.804394  ...    5.250   
        2018-03-01 04:00:00  89.900002  3.137333  1.153006  ...   14.500   
        2018-03-01 05:00:00  90.800003  4.118162  1.164394  ...   50.125   
        2018-03-01 06:00:00  91.199997  4.183517 -0.159279  ...    0.875   
        2018-03-01 07:00:00  89.800003  3.131047 -0.425308  ...  100.000   
...                                ...       ...       ...  ...      ...   
YORK    2018-03-31 22:00:00  61.400002  1.582819  4.688375  ...  100.000   
        2018-03-31 23:00:00  65.199997  1.153378  5.039130  ...   66.000   
        2018-04-01 00:00:00  52.700001  2.751124  5.637678  ...  100.000   
        2018-04-01 01:00:00  63.599998  3.357752  6.038958  ...   94.500   
        2018-04-01 02:00:00  70.199997  3.011552  8.184587  ...    5.000   

                             asnow  cape      dswrf       dlwrf           gh  \
station valid_time                                                             
ADDI    2018-03-01 03:00:00    0.0   0.0   0.000000  272.700012  5531.544434   
        2018-03-01 04:00:00    0.0   0.0   0.000000  270.700012  5524.548828   
        2018-03-01 05:00:00    0.0   0.0   0.000000  269.899994  5527.829102   
        2018-03-01 06:00:00    0.0   0.0   0.000000  268.399994  5532.908203   
        2018-03-01 07:00:00    0.0  10.0   0.000000  273.200012  5534.237793   
...                            ...   ...        ...         ...          ...   
YORK    2018-03-31 22:00:00    0.0   0.0  35.299999  320.600006  5482.334961   
        2018-03-31 23:00:00    0.0  30.0  46.700001  303.200012  5480.329102   
        2018-04-01 00:00:00    0.0   0.0   0.000000  299.600006  5477.961914   
        2018-04-01 01:00:00    0.0   0.0   0.000000  278.799988  5469.405762   
        2018-04-01 02:00:00    0.0   0.0   0.000000  264.299988  5472.545410   

                              u_total       u_dir  lead time  lsm  
station valid_time                                           

In [ ]:
stations = pd.read_csv("/home/aevans/nwp_bias/src/correlation/data/nlcd_hrrr.csv")[
    "station"
]
stations

In [56]:
final_df_list = []  # Store DataFrames here

for s in stations:
    df = forecast_error.hrrr_error("01", s, "t2m")

    # Aggregate monthly mean target error
    months_df = (
        df.groupby([df.valid_time.dt.month, "station"])["target_error"]
        .mean()
        .reset_index()
        .rename(columns={"valid_time": "month"})  # Rename for clarity
    )

    final_df_list.append(months_df)  # Store result

# Concatenate all results into a single DataFrame
final_df = pd.concat(final_df_list, ignore_index=True)

# Re-group after concatenation to ensure correct aggregation
final_df = final_df.groupby(["month", "station"])["target_error"].mean().reset_index()

In [ ]:
df

In [ ]:
lulc = pd.read_csv("/home/aevans/nwp_bias/src/correlation/data/nlcd_gfs.csv")
lulc = lulc.drop(columns=["site", "station"])
keys = [
    "Open Water",
    "Developed, Open",
    "Developed, Low",
    "Developed, Medium",
    "Developed High",
    "Barren Land",
    "Deciduous Forest",
    "Evergreen Forest",
    "Mixed Forest",
    "Shrub/Scrub",
    "Grassland/Herbaceous",
    "Pasture/Hay",
    "Cultivated Crops",
    "Woody Wetlands",
    "Emergent Herbaceous Wetlands",
    "OOA",
]

In [ ]:
lulc

In [ ]:
months_df

# GFS

In [ ]:
(
    df_pers,
    df_rho,
    df_tau,
    df_p_score,
    df_p_score_rho,
    df_p_score_tau,
) = get_corrs.get_corrs(months_df, lulc, keys, "target_error")

In [ ]:
# df_pers = get_corrs.rename_columns(df_pers)
df_pers.keys()

In [ ]:
plot_heatmaps.plot_heatmap_corrs(df_pers, "PERS", "NLCD", "GFS")

In [ ]:
plot_heatmaps.plot_heatmap_corrs(df_rho, "RHO", "NLCD", "GFS")

In [ ]:
plot_heatmaps.plot_heatmap_corrs(df_tau, "TAU", "NLCD", "GFS")

In [ ]:
dec_df = months_df[months_df["time"] == 12]
dec_percentage = lulc["22"].tolist()
dec_error = dec_df["t2m_error"].tolist()
plt.figure(figsize=(15, 12))
plt.scatter(
    dec_percentage,
    dec_error,
    c=dec_percentage,
    s=50,
    edgecolors="black",
    cmap="Blues",
    vmin=0,
    vmax=100,
)
plt.title("Percent of Developed, Low Class in Buffer v T2M Error for December")
plt.xlabel("Percent of Developed, Low Class in buffer")
plt.ylabel("T2M Error")
plt.ylim(-2.0, 2.0)
plt.colorbar()

In [ ]:
jan_df = months_df[months_df["time"] == 1]
jan_percentage = lulc["42"].tolist()
jan_error = jan_df["t2m_error"].tolist()
plt.figure(figsize=(15, 12))
plt.scatter(
    jan_percentage,
    jan_error,
    c=jan_percentage,
    s=50,
    edgecolors="black",
    cmap="Greens",
    vmin=0,
    vmax=100,
)
plt.title("Percent of Evergreen Forest Class in Buffer v T2M Error for January")
plt.xlabel("Percent of Evergreen Forest Class in buffer")
plt.ylabel("T2M Error")
plt.ylim(-2.0, 2.0)
plt.colorbar()

In [ ]:
jun_df = months_df[months_df["time"] == 6]
jun_percentage = lulc["82"].tolist()
jun_error = jun_df["t2m_error"].tolist()
plt.figure(figsize=(15, 12))
plt.scatter(
    jun_percentage,
    jun_error,
    c=jun_percentage,
    s=50,
    edgecolors="black",
    cmap="Oranges",
    vmin=0,
    vmax=100,
)
plt.title("Percent of Cultivated Class in Buffer v T2M Error for June")
plt.xlabel("Percent of Cultivated Crops Class in buffer")
plt.ylabel("T2M Error")
plt.ylim(-2.0, 2.0)
plt.colorbar()